In [ ]:
import json
import urllib.request

In [ ]:
NUM_OF_POSTS = 900

In [ ]:
import time
hdr = {'User-Agent': 'osx:r/relationships.single.result:v1.0 (by /u/python_machine_learn)'}
url = 'https://www.reddit.com/r/relationships/top/.json?sort=top&t=all&limit=100' 
req = urllib.request.Request(url, headers = hdr)
text_data = urllib.request.urlopen(req).read()
text_data_json = text_data.decode('utf8')
data = list(json.loads(text_data_json).values())
posts_all = data[0]['children']
while (len(posts_all) < NUM_OF_POSTS):
    time.sleep(2)
    last = posts_all[-1]['data']['name']
    url = 'https://www.reddit.com/r/relationships/top/.json?sort=top&t=all&limit=100&after=%s' % last 
    req = urllib.request.Request(url, headers = hdr)
    text_data = urllib.request.urlopen(req).read()
    text_data_json = text_data.decode('utf8')
    data = list(json.loads(text_data_json).values())
    post = data[0]['children']
    posts_all += post

In [ ]:
print(len(posts_all))

In [ ]:
post_title = []
post_flair = []
post_date = []
post_numComments = []
post_score = []

In [ ]:
for k in range(0, NUM_OF_POSTS):
    post_title.append(posts_all[k]['data']['title'])
    post_flair.append(posts_all[k]['data']['link_flair_text'])
    post_date.append(posts_all[k]['data']['created_utc'])
    post_numComments.append(posts_all[k]['data']['num_comments'])
    post_score.append(posts_all[k]['data']['score'])

In [ ]:
import pandas as pd
from pandas import DataFrame
posts_df = DataFrame({'Date': post_date,
                       'Title': post_title,
                        'Flair' : post_flair,
                        'Comments' : post_numComments,
                         'Score' : post_score
                     })
posts_df = posts_df[['Date', 'Title', 'Flair', 'Comments', 'Score']]

In [ ]:
posts_df.head()